In [111]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from numpy.linalg import norm
from string import digits
import pickle

data = pd.read_csv('true.csv', error_bad_lines=False);
data_text = data[['text']]
data_text['index'] = data_text.index #creating a new cloumn index.
documents = data_text

print(len(documents))

documents.tail()

C:\Users\afzal\AppData\Local\Temp\ipykernel_16400\1903849182.py:8: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('true.csv', error_bad_lines=False);


21417


C:\Users\afzal\AppData\Local\Temp\ipykernel_16400\1903849182.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_text['index'] = data_text.index #creating a new cloumn index.


,text,index
21412,BRUSSELS (Reuters) - NATO allies on Tuesday we...,21412
21413,"LONDON (Reuters) - LexisNexis, a provider of l...",21413
21414,MINSK (Reuters) - In the shadow of disused Sov...,21414
21415,MOSCOW (Reuters) - Vatican Secretary of State ...,21415
21416,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,21416


In [3]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import numpy as np
np.random.seed(2018)

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\afzal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
stemmer = SnowballStemmer('english')

In [4]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v')) # stemmer was not defined soo top code fixes it

def preprocess (text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len (token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [5]:
doc_sample = documents[documents['index'] == 20].values[0][0]
# print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
# print(words)
print('\n\n tokenized and lemmatized document: ')
# print(preprocess(doc_sample))



 tokenized and lemmatized document: 


In [6]:
processed_docs = documents['text'].map(preprocess)

In [7]:
processed_docs[:10]

0    [washington, reuter, head, conserv, republican...
1    [washington, reuter, transgend, peopl, allow, ...
2    [washington, reuter, special, counsel, investi...
3    [washington, reuter, trump, campaign, advis, g...
4    [seattl, washington, reuter, presid, donald, t...
5    [west, palm, beach, washington, reuter, white,...
6    [west, palm, beach, reuter, presid, donald, tr...
7    [follow, statement, post, verifi, twitter, acc...
8    [follow, statement, post, verifi, twitter, acc...
9    [washington, reuter, alabama, secretari, state...
Name: text, dtype: object

In [8]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 action
1 administr
2 agre
3 aid
4 approach
5 approv
6 arriv
7 assist
8 balloon
9 battl
10 begin


In [9]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)


In [10]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[21416]

[(13, 1),
 (36, 2),
 (57, 1),
 (58, 1),
 (66, 1),
 (68, 2),
 (70, 1),
 (129, 1),
 (204, 2),
 (212, 1),
 (221, 1),
 (250, 1),
 (265, 1),
 (324, 1),
 (346, 1),
 (347, 1),
 (359, 1),
 (434, 3),
 (474, 1),
 (482, 1),
 (499, 1),
 (530, 1),
 (540, 1),
 (577, 2),
 (598, 1),
 (620, 1),
 (656, 2),
 (703, 2),
 (716, 1),
 (743, 1),
 (752, 1),
 (767, 3),
 (794, 4),
 (839, 1),
 (868, 1),
 (895, 1),
 (967, 2),
 (1070, 1),
 (1124, 1),
 (1320, 1),
 (1384, 1),
 (1419, 1),
 (1425, 1),
 (1482, 1),
 (1541, 1),
 (1615, 1),
 (1744, 1),
 (1775, 2),
 (1825, 1),
 (1988, 1),
 (2050, 1),
 (2092, 1),
 (2109, 1),
 (2236, 1),
 (2263, 1),
 (2275, 4),
 (2276, 3),
 (2441, 1),
 (2653, 1),
 (2821, 1),
 (2947, 1),
 (3009, 1),
 (3054, 1),
 (3264, 2),
 (3268, 1),
 (3665, 1),
 (3813, 2),
 (4026, 1),
 (4205, 1),
 (4630, 1),
 (5236, 1),
 (5496, 2),
 (5682, 1),
 (5769, 1),
 (6147, 3),
 (6333, 1),
 (7755, 1)]

In [11]:
bow_doc_997 = bow_corpus[997]

for i in range(len(bow_doc_997)):
        print("Word {} (\"{}\") appears {} time.".format(bow_doc_997[i][0], 
                                               dictionary[bow_doc_997[i][0]],
                                                         bow_doc_997[i][1]))

Word 36 ("countri") appears 1 time.
Word 53 ("donald") appears 1 time.
Word 86 ("govern") appears 1 time.
Word 129 ("militari") appears 1 time.
Word 131 ("month") appears 1 time.
Word 144 ("peopl") appears 1 time.
Word 170 ("respons") appears 1 time.
Word 172 ("return") appears 1 time.
Word 192 ("speak") appears 1 time.
Word 198 ("support") appears 1 time.
Word 206 ("trump") appears 1 time.
Word 209 ("urg") appears 2 time.
Word 213 ("washington") appears 1 time.
Word 224 ("access") appears 1 time.
Word 227 ("allow") appears 2 time.
Word 254 ("depart") appears 3 time.
Word 265 ("forc") appears 1 time.
Word 300 ("major") appears 1 time.
Word 334 ("right") appears 1 time.
Word 339 ("secretari") appears 1 time.
Word 340 ("senior") appears 1 time.
Word 347 ("statement") appears 2 time.
Word 407 ("investig") appears 1 time.
Word 410 ("launch") appears 1 time.
Word 429 ("report") appears 1 time.
Word 450 ("alleg") appears 1 time.
Word 482 ("includ") appears 1 time.
Word 498 ("respond") appear

In [12]:
from gensim import corpora, models

In [13]:
tfidf = models.TfidfModel (bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.026841685091095916),
 (1, 0.040874722914628694),
 (2, 0.02835365174741251),
 (3, 0.03660473880060063),
 (4, 0.039605125068353925),
 (5, 0.03092179071764382),
 (6, 0.03897867535244838),
 (7, 0.08024173683071609),
 (8, 0.0740943853428846),
 (9, 0.03788393586016424),
 (10, 0.027398418040369982),
 (11, 0.035840612711428195),
 (12, 0.04961255800869147),
 (13, 0.05782661322801332),
 (14, 0.03974153302302116),
 (15, 0.0291238513421231),
 (16, 0.0616129444077586),
 (17, 0.07687915789702442),
 (18, 0.02705966268183348),
 (19, 0.17385661598683122),
 (20, 0.041903357918051526),
 (21, 0.03124372753859764),
 (22, 0.01827310221513898),
 (23, 0.053251745351518935),
 (24, 0.031618017464307764),
 (25, 0.022935188852024425),
 (26, 0.06781583839128565),
 (27, 0.03762650186346645),
 (28, 0.051686343451263946),
 (29, 0.025333152711866598),
 (30, 0.017199206457189753),
 (31, 0.04758258500547958),
 (32, 0.06620182303451502),
 (33, 0.08780959236607397),
 (34, 0.04958184699935254),
 (35, 0.0422453453171

In [14]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10,id2word=dictionary, passes=2, workers=2)

In [15]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.024*"parti" + 0.016*"elect" + 0.012*"vote" + 0.007*"percent" + 0.007*"year" + 0.007*"poll" + 0.007*"court" + 0.007*"trump" + 0.006*"support" + 0.006*"rule"
Topic: 1 
Words: 0.043*"trump" + 0.009*"russian" + 0.008*"russia" + 0.007*"republican" + 0.007*"washington" + 0.007*"donald" + 0.007*"hous" + 0.007*"white" + 0.006*"offici" + 0.006*"putin"
Topic: 2 
Words: 0.012*"minist" + 0.009*"european" + 0.008*"govern" + 0.006*"britain" + 0.006*"deal" + 0.006*"trump" + 0.006*"countri" + 0.006*"year" + 0.005*"tell" + 0.005*"leader"
Topic: 3 
Words: 0.008*"year" + 0.008*"percent" + 0.008*"trump" + 0.006*"peopl" + 0.006*"govern" + 0.006*"hurrican" + 0.006*"million" + 0.005*"irma" + 0.005*"florida" + 0.005*"island"
Topic: 4 
Words: 0.007*"militari" + 0.007*"polic" + 0.007*"peopl" + 0.007*"myanmar" + 0.007*"attack" + 0.007*"year" + 0.006*"govern" + 0.006*"kill" + 0.006*"secur" + 0.006*"rohingya"
Topic: 5 
Words: 0.026*"north" + 0.023*"korea" + 0.012*"nuclear" + 0.009*"unit" + 0.008

In [16]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=20, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.014*"maduro" + 0.012*"venezuela" + 0.007*"venezuelan" + 0.005*"trump" + 0.004*"caraca" + 0.004*"nicola" + 0.003*"republican" + 0.003*"unesco" + 0.003*"senat" + 0.003*"scottish"
Topic: 1 Word: 0.010*"macron" + 0.007*"cuba" + 0.006*"merkel" + 0.005*"kurz" + 0.005*"poland" + 0.005*"cuban" + 0.005*"european" + 0.004*"euro" + 0.004*"french" + 0.004*"mogadishu"
Topic: 2 Word: 0.007*"merkel" + 0.006*"saudi" + 0.006*"pakistan" + 0.004*"coalit" + 0.004*"india" + 0.004*"qatar" + 0.004*"german" + 0.004*"germani" + 0.003*"hurrican" + 0.003*"arabia"
Topic: 3 Word: 0.004*"refuge" + 0.004*"trump" + 0.004*"asylum" + 0.003*"republican" + 0.003*"australia" + 0.003*"senat" + 0.003*"seeker" + 0.003*"hous" + 0.002*"immigr" + 0.002*"polic"
Topic: 4 Word: 0.012*"israel" + 0.012*"palestinian" + 0.011*"jerusalem" + 0.010*"ireland" + 0.008*"isra" + 0.006*"irish" + 0.005*"britain" + 0.005*"brexit" + 0.005*"netanyahu" + 0.004*"northern"
Topic: 5 Word: 0.012*"temer" + 0.005*"brazil" + 0.005*"babi"

In [17]:
processed_docs[4310]

['moscow',
 'reuter',
 'russia',
 'say',
 'hop',
 'forthcom',
 'talk',
 'moscow',
 'secretari',
 'state',
 'tillerson',
 'product',
 'say',
 'extrem',
 'worri',
 'unit',
 'state',
 'decid',
 'unilater',
 'attack',
 'north',
 'korea',
 'north',
 'korea',
 'emerg',
 'press',
 'foreign',
 'polici',
 'problem',
 'presid',
 'donald',
 'trump',
 'pyongyang',
 'conduct',
 'nuclear',
 'test',
 'work',
 'develop',
 'nuclear',
 'tip',
 'missil',
 'reach',
 'unit',
 'state',
 'offici',
 'tell',
 'reuter',
 'weekend',
 'navi',
 'strike',
 'group',
 'nuclear',
 'power',
 'aircraft',
 'carrier',
 'steam',
 'western',
 'pacif',
 'forc',
 'tillerson',
 'visit',
 'moscow',
 'secretari',
 'state',
 'hold',
 'talk',
 'sergei',
 'lavrov',
 'russian',
 'counterpart',
 'wednesday',
 'time',
 'russia',
 'tie',
 'languish',
 'post',
 'cold',
 'russian',
 'foreign',
 'ministri',
 'statement',
 'ahead',
 'visit',
 'say',
 'concern',
 'aspect',
 'foreign',
 'polici',
 'includ',
 'libya',
 'yemen',
 'syria',
 'sa

In [18]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.3875274658203125	 
Topic: 0.026*"north" + 0.023*"korea" + 0.012*"nuclear" + 0.009*"unit" + 0.008*"missil" + 0.008*"china" + 0.008*"sanction" + 0.007*"korean" + 0.007*"govern" + 0.006*"test"

Score: 0.37800100445747375	 
Topic: 0.043*"trump" + 0.009*"russian" + 0.008*"russia" + 0.007*"republican" + 0.007*"washington" + 0.007*"donald" + 0.007*"hous" + 0.007*"white" + 0.006*"offici" + 0.006*"putin"

Score: 0.22995060682296753	 
Topic: 0.009*"govern" + 0.009*"syria" + 0.008*"iran" + 0.008*"syrian" + 0.008*"forc" + 0.008*"islam" + 0.007*"unit" + 0.007*"saudi" + 0.006*"group" + 0.005*"iraq"


In [19]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.7459374070167542	 
Topic: 0.015*"korea" + 0.013*"north" + 0.008*"nuclear" + 0.007*"korean" + 0.007*"missil" + 0.006*"china" + 0.005*"south" + 0.005*"test" + 0.005*"japan" + 0.004*"pyongyang"

Score: 0.14362655580043793	 
Topic: 0.005*"parti" + 0.003*"catalan" + 0.003*"britain" + 0.003*"trump" + 0.003*"parliament" + 0.003*"european" + 0.003*"minist" + 0.003*"percent" + 0.003*"elect" + 0.003*"china"

Score: 0.10494652390480042	 
Topic: 0.015*"turkey" + 0.013*"syrian" + 0.011*"syria" + 0.010*"turkish" + 0.009*"erdogan" + 0.008*"ankara" + 0.007*"islam" + 0.006*"deir" + 0.006*"russian" + 0.006*"raqqa"


In [90]:
heading = 'Agreement between President and basil Rajapaksha'
topic_list_head = ''
def lda_model_head (heading):
    bow_vector = dictionary.doc2bow(preprocess(heading))
    count=0
    for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
        if count==0:
            topic_list_head=lda_model.print_topic(index, 10)
            print(topic_list_head)
            count=count+1
#         print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 10)))
    ret_head = topic_list_head
    return ret_head
top_head = lda_model_head(heading)

0.012*"minist" + 0.009*"european" + 0.008*"govern" + 0.006*"britain" + 0.006*"deal" + 0.006*"trump" + 0.006*"countri" + 0.006*"year" + 0.005*"tell" + 0.005*"leader"


In [91]:
# below is whee the model will be called by user input

In [92]:


remove_digits = str.maketrans('', '', digits)
print(top_head)
    

0.012*"minist" + 0.009*"european" + 0.008*"govern" + 0.006*"britain" + 0.006*"deal" + 0.006*"trump" + 0.006*"countri" + 0.006*"year" + 0.005*"tell" + 0.005*"leader"


In [93]:

def best_head (top_head):
    rest = top_head.translate(remove_digits)
    final_list_head = re.sub(r'[^\w\s]', '', rest)
    flh = final_list_head.split()
    return flh

best_head_final = best_head(top_head)
print(best_head1)

['minist', 'european', 'govern', 'britain', 'deal', 'trump', 'countri', 'year', 'tell', 'leader']


In [128]:
content = 'Close on the heels of the project that translocated cheetahs from Namibia, the Indian government is considering a proposal from Colombo to export a number of gaurs, or Indian bisons, to Sri Lanka to revive the population of gavaras that have been extinct in the island since the end of the 17th century.'
topic_list_content = ''
def lad_model_content(content):
    bow_vector = dictionary.doc2bow(preprocess(content))
    count=0
    for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
        if count==0:
            topic_list_content=lda_model.print_topic(index, 10)
            count=count+1
        print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 10)))
    ret_content = topic_list_content
    return ret_content
top_content = lad_model_content(content)

Score: 0.9399711489677429	 Topic: 0.008*"year" + 0.008*"percent" + 0.008*"trump" + 0.006*"peopl" + 0.006*"govern" + 0.006*"hurrican" + 0.006*"million" + 0.005*"irma" + 0.005*"florida" + 0.005*"island"


In [105]:
def best_content(top_content):
    rest = top_content.translate(remove_digits)
    final_list_content = re.sub(r'[^\w\s]', '', rest)
    print(final_list_content.split())
    flc = (final_list_content.split())
# flc = convert(final_list_content.split())
    return flc
best_content_final = best_content(top_content)

['year', 'percent', 'trump', 'peopl', 'govern', 'hurrican', 'million', 'irma', 'florida', 'island']


In [106]:

# Cosine will be taken from here for both heading and content

In [107]:
sw = stopwords.words('english')
l1 =[];l2 =[]

In [108]:
x_set = {w for w in best_head_final if not w in sw}
y_set = {w for w in best_content_final if not w in sw}
print(x_set)
print (y_set)

{'minist', 'leader', 'european', 'tell', 'countri', 'britain', 'year', 'deal', 'govern', 'trump'}
{'peopl', 'island', 'hurrican', 'percent', 'florida', 'million', 'irma', 'year', 'govern', 'trump'}


In [109]:
rvector = x_set.union(y_set) 
for w in rvector:
    if w in best_head_final: l1.append(1) # create a vector
    else: l1.append(0)
    if w in best_content_final: l2.append(1)
    else: l2.append(0)
c = 0

In [110]:
for i in range(len(rvector)):
        c+= l1[i]*l2[i]
cosine = c / float((sum(l1)*sum(l2))**0.5)
print("similarity: ", cosine)

similarity:  0.3


In [1]:
#  hwo to get the accuracy of the LDA model
#  train the dataset inlucding sri lanka news
# 

In [2]:
# export the model using the pickle

In [117]:
pickle.dump(top_content, open('LdaModel', 'wb'))

In [119]:
test_10 = pickle.load(open('LdaModel', 'rb'))
test_10.top_content(content)

AttributeError: 'str' object has no attribute 'top_content'

In [130]:
data = {"model": top_content}
with open ('saved_steps.pkl' , 'wb') as file:
    pickle.dump(data,file)

In [125]:
with open ('saved_steps.pkl', 'rb') as file:
    data = pickle.load(file)

load_model = data["model"]

In [131]:
y_pred = load_model.top_content(content)

AttributeError: 'function' object has no attribute 'top_content'